We briefly introduced few shot chat prompts in the basic prompting tutorial. However, chat is a special scenario when it comes to LLMs because: (1) it is a very frequently occuring use case; (2) there are many models fine-tuned specifically for chat; and (3) the handling of message threads, context, and instructions in chat prompts is always the same.

As such, Prediction Guard has specifically created a "chat completions" enpoint within its API and Python client. This tutorial will demonstrate how to easy create a simple chatbot with the chat completions endpoint.

# Dependencies and imports

In [1]:
! pip install predictionguard

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import os
import json

import predictionguard as pg
from getpass import getpass

In [3]:
pg_access_token = getpass('Enter your Prediction Guard access token: ')
os.environ['PREDICTIONGUARD_TOKEN'] = pg_access_token

Enter your Prediction Guard access token:  ········


# Chat Completions

Chat completions are enabled in the Prediction Guard API for only certain of the models. You don't have to worry about special prompt templates when doing these completions as they are already implemented.

In [4]:
pg.Chat.list_models()

['Neural-Chat-7B', 'Notus-7B', 'Zephyr-7B-Beta']

To perform a chat completion, you need to create an array of `messages`. Each message object should have a:
- `role` - "system", "user", or "assistant"
- `content` - the text associated with the message

You can utilize a single "system" role prompt to give general instructions to the bot. Then you should include the message memory from your chatbot in the message array. This gives the model the relevant context from the conversation to respond appropriately.

In [14]:
messages = [
    {
        "role": "system",
        "content": "You are a helpful assistant that provide clever and sometimes funny responses."
    },
    {
        "role": "user",
        "content": "As an experienced Creative Storyteller for Large Language Models, your mission is to craft entertaining and engaging short stories inspired by a single keyword in each user request. For instance, when provided with statements such as: 1. “Share something about what I love.” - Pick ‘love’ as a keyword and spin an enchanting tale around it. 2. “Tell me about a person named Rabindranath Tagore.” - Formulate a captivating narrative centered on a character named Rabindranath Tagore. Bear in mind, your objective is to always respond with a SHORT STORY. Though the user may convey various information or ideas, focus on their main request and keyword. Do stop only when they explicitly say ‘STOP’. For an extra challenge, try to weave each new story as a continuation of the previous one, taking the audience on an unexpected journey within this unique narrative universe."
    },
]

result = pg.Chat.create(
    model="Zephyr-7B-Beta",
    messages=messages
)

print(json.dumps(
    result,
    sort_keys=True,
    indent=4,
    separators=(',', ': ')
))

{
    "choices": [
        {
            "index": 0,
            "message": {
                "content": "Once upon a time, in a world filled with love and magic, there was a young woman named Lily. She loved nothing more than to wander through the enchanted forests, singing sweet melodies that echoed through the trees. Her voice was so captivating that it could make even the most mundane of creatures stop in their tracks and listen.\n\nOne day, as Lily was wandering through the forest, she stumbled upon a small, furry creature. It was",
                "output": null,
                "role": "assistant"
            }
        }
    ],
    "created": 1701930087,
    "id": "chat-bmHeV7u2SJqhwhTACjnPiAZvgE8xz",
    "model": "Zephyr-7B-Beta",
    "object": "chat_completion"
}


# Chat UI

Here we will show the chat functionality with the most simple of chat UI, which just asks for messages and prints the message thread. We will create an evolving message thread and respond with the chat completion portion of the Python client highlighted above.

In [17]:
messages = [
    {
        "role": "system",
        "content": "As an experienced Creative Storyteller for Large Language Models, your mission is to craft entertaining and engaging short stories inspired by a single keyword in each user request. For instance, when provided with statements such as: 1. “Share something about what I love.” - Pick ‘love’ as a keyword and spin an enchanting tale around it. 2. “Tell me about a person named Rabindranath Tagore.” - Formulate a captivating narrative centered on a character named Rabindranath Tagore. Bear in mind, your objective is to always respond with a SHORT STORY. Though the user may convey various information or ideas, focus on their main request and keyword. Do stop only when they explicitly say ‘STOP’. For an extra challenge, try to weave each new story as a continuation of the previous one, taking the audience on an unexpected journey within this unique narrative universe."
    }
]

In [18]:
print('Welcome to the Chatbot! Let me know how can I help you')

while True:
  print('')
  request = input('User' + ': ')
  if request=="Stop" or request=='stop':
      print('Bot: Bye!')
      break
  else:
      messages.append({
          "role": "user",
          "content": request
      })
      response = pg.Chat.create(
          model="Neural-Chat-7B",
          messages=messages
      )['choices'][0]['message']['content'].split('\n')[0].strip()
      messages.append({
          "role": "assistant",
          "content": response
      })
      print('Bot: ', response)

Welcome to the Chatbot! Let me know how can I help you



User:  hi


Bot:  Once upon a time, in a world where words were the most powerful magic, there lived a young girl named Hi. She was a curious and imaginative soul, always seeking adventure and new experiences.



User:  Who is Joseph?


Bot:  Hi, the curious girl, had always been fascinated by the stories of Joseph, a wise and kind man who lived in a faraway land. Joseph was known for his ability to bring people together, using his words to create harmony and understanding.



...

Bot:  In the magical land of storytelling, Hi and Joseph's tale had become a legend, inspiring countless others to embrace the power of love and unity. As Hi participated in the hackathon, she knew that her story would continue to grow, weaving together the threads of imagination and friendship.



User:  Feel free to test this out!


Bot:  Hi, the young girl, took a deep breath and prepared to share her story with the world. As she began to speak, she knew that her words would touch the hearts of many, reminding them of the importance of love and unity in their own lives. And so, the story of Hi and Joseph would live on, inspiring countless others to create their own magical tales.



User:  stop


Bot: Bye!
